### Quick Start

The `ssms` package serves two purposes. 

1. Easy access to *fast simulators of sequential sampling models*
   
2. Support infrastructure to construct training data for various approaches to likelihood / posterior amortization

We provide two minimal examples here to illustrate how to use each of the two capabilities.


#### Install 

Let's start with *installing* the `ssms` package.

You can do so by typing,

`pip install git+https://github.com/AlexanderFengler/ssm_simulators`

in your terminal.

Below you find a basic tutorial on how to use the package.

#### Tutorial

In [1]:
# Import necessary packages
import numpy as np
import pandas as pd
import ssms

#### Using the Simulators

Let's start with using the basic simulators. 
You access the main simulators through the  `ssms.basic_simulators.simulator` function.

To get an idea about the models included in `ssms`, use the `config` module.
The central dictionary with metadata about included models sits in `ssms.config.model_config`. 

In [2]:
# Check included models
list(ssms.config.model_config.keys())[:10]

['ddm',
 'ddm_legacy',
 'angle',
 'weibull',
 'levy',
 'levy_angle',
 'full_ddm',
 'gamma_drift',
 'gamma_drift_angle',
 'ds_conflict_drift']

In [3]:
# Take an example config for a given model
ssms.config.model_config["ddm"]

{'name': 'ddm',
 'params': ['v', 'a', 'z', 't'],
 'param_bounds': [[-3.0, 0.3, 0.1, 0.0], [3.0, 2.5, 0.9, 2.0]],
 'boundary_name': 'constant',
 'boundary': <function ssms.basic_simulators.boundary_functions.constant(t=0)>,
 'boundary_params': [],
 'n_params': 4,
 'default_params': [0.0, 1.0, 0.5, 0.001],
 'nchoices': 2,
 'simulator': <cyfunction ddm_flexbound at 0x12a28de50>}

In [4]:
ssms.config.model_config["weibull"]["params"]

['v', 'a', 'z', 't', 'alpha', 'beta']

**Note:**
The usual structure of these models includes,

- Parameter names (`'params'`)
- Bounds on the parameters (`'param_bounds'`)
- A function that defines a boundary for the respective model (`'boundary'`)
- The number of parameters (`'n_params'`)
- Defaults for the parameters (`'default_params'`)
- The number of choices the process can produce (`'nchoices'`)

The `'hddm_include'` key concerns information useful for integration with the [hddm](https://github.com/hddm-devs/hddm) python package, which facilitates hierarchical bayesian inference for sequential sampling models. It is not important for the present tutorial.

In [5]:
from ssms.basic_simulators.simulator import simulator

sim_out = simulator(
    model="weibull",
    theta={"v": 0.0, "a": 1.65, "z": 0.5, "t": 1.0005, "alpha": 0.6, "beta": 2.0},
    n_samples=1000,
    smooth_unif=False,
)

The output of the simulator is a `dictionary` with three elements.

1. `rts` (array)
2. `choices` (array)
3. `metadata` (dictionary)

The `metadata` includes the named parameters, simulator settings, and more.

#### Using the Training Data Generators

The training data generators sit on top of the simulator function to turn raw simulations into usable training data for training machine learning algorithms aimed at posterior or likelihood armortization.

We will use the `data_generator` class from `ssms.dataset_generators`. Initializing the `data_generator` boils down to supplying two configuration dictionaries.

1. The `generator_config`, concerns choices as to what kind of training data one wants to generate.
2. The `model_config` concerns choices with respect to the underlying generative *sequential sampling model*. 

We will consider a basic example here, concerning data generation to prepare for training [LANs](https://elifesciences.org/articles/65074).

Let's start by peeking at an example `generator_config`.

In [6]:
ssms.config.data_generator_config["lan"]

{'output_folder': 'data/lan_mlp/',
 'dgp_list': 'ddm',
 'nbins': 0,
 'n_samples': 100000,
 'n_parameter_sets': 10000,
 'n_parameter_sets_rejected': 100,
 'n_training_samples_by_parameter_set': 1000,
 'max_t': 20.0,
 'delta_t': 0.001,
 'pickleprotocol': 4,
 'n_cpus': 'all',
 'kde_data_mixture_probabilities': [0.8, 0.1, 0.1],
 'simulation_filters': {'mode': 20,
  'choice_cnt': 0,
  'mean_rt': 17,
  'std': 0,
  'mode_cnt_rel': 0.95},
 'negative_rt_cutoff': -66.77497,
 'n_subruns': 10,
 'bin_pointwise': False,
 'separate_response_channels': False,
 'smooth_unif': True}

You usually have to make just few changes to this basic configuration dictionary.
An example below.

In [7]:
from copy import deepcopy

# Initialize the generator config (for MLP LANs)
generator_config = deepcopy(ssms.config.data_generator_config["lan"])
# Specify generative model (one from the list of included models mentioned above)
generator_config["dgp_list"] = "ddm"
# Specify number of parameter sets to simulate
generator_config["n_parameter_sets"] = 1000
# Specify how many samples a simulation run should entail
generator_config["n_samples"] = 2000
generator_config["n_cpus"] = "all"
generator_config["smooth_unif"] = True

Now let's define our corresponding `model_config`.

In [8]:
model_config = ssms.config.model_config["ddm"]
print(model_config)

{'name': 'ddm', 'params': ['v', 'a', 'z', 't'], 'param_bounds': [[-3.0, 0.3, 0.1, 0.0], [3.0, 2.5, 0.9, 2.0]], 'boundary_name': 'constant', 'boundary': <function constant at 0x10a91d1b0>, 'boundary_params': [], 'n_params': 4, 'default_params': [0.0, 1.0, 0.5, 0.001], 'nchoices': 2, 'simulator': <cyfunction ddm_flexbound at 0x12a28de50>}


We are now ready to initialize a `data_generator`, after which we can generate training data using the `generate_data_training_uniform` function, which will use the hypercube defined by our parameter bounds from the `model_config` to uniformly generate parameter sets and corresponding simulated datasets.

In [9]:
my_dataset_generator = ssms.dataset_generators.lan_mlp.data_generator(
    generator_config=generator_config, model_config=model_config
)

n_cpus used:  12
checking:  data/lan_mlp/


In [10]:
training_data = my_dataset_generator.generate_data_training_uniform(save=False)

simulation round: 1  of 10


simulation round: 2  of 10
simulation round: 3  of 10
simulation round: 4  of 10
simulation round: 5  of 10
simulation round: 6  of 10
simulation round: 7  of 10
simulation round: 8  of 10
simulation round: 9  of 10
simulation round: 10  of 10


In [11]:
training_data

{'cpn_data': array([[ 0.61326325,  1.1373361 ,  0.50439495,  0.15703481],
        [-1.1348734 ,  1.1063089 ,  0.7927923 ,  0.9796527 ],
        [ 0.70365846,  2.3286552 ,  0.419938  ,  1.0666826 ],
        ...,
        [ 0.7264401 ,  0.7720335 ,  0.421435  ,  0.18856497],
        [-0.11527749,  1.8669481 ,  0.52418655,  0.42642087],
        [ 0.48887515,  1.5738922 ,  0.14552219,  0.3938222 ]],
       dtype=float32),
 'cpn_labels': array([0.1865, 0.6685, 0.0725, 0.7615, 0.926 , 0.603 , 0.3265, 0.074 ,
        0.0905, 0.0025, 0.042 , 0.001 , 0.14  , 0.177 , 0.9615, 0.732 ,
        0.014 , 0.937 , 0.0315, 0.638 , 0.502 , 0.9365, 0.2035, 0.511 ,
        0.0105, 0.855 , 0.7605, 0.06  , 0.815 , 0.002 , 0.0215, 0.994 ,
        0.9825, 0.97  , 0.2135, 0.849 , 0.2415, 0.5165, 0.7995, 0.8615,
        0.189 , 0.041 , 0.39  , 0.751 , 0.952 , 0.115 , 0.8825, 0.921 ,
        0.9935, 0.529 , 0.275 , 0.0525, 0.3135, 0.004 , 0.0045, 0.029 ,
        0.1765, 0.8295, 0.0465, 0.0545, 0.004 , 0.737 , 0.182

`training_data` is a dictionary containing four keys:

1. `data` the features for [LANs](https://elifesciences.org/articles/65074), containing vectors of *model parameters*, as well as *rts* and *choices*.
2. `labels` which contain approximate likelihood values
3. `generator_config`, as defined above
4. `model_config`, as defined above

You can now use this training data for your purposes. If you want to train [LANs](https://elifesciences.org/articles/65074) yourself, you might find the [LANfactory](https://github.com/AlexanderFengler/LANfactory) package helpful.

You may also simply find the basic simulators provided with the **ssms** package useful, without any desire to use the outputs into training data for amortization purposes.

##### END